[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/query_transformations.ipynb)

# 改进RAG系统检索的查询转换

## 概述

此代码实现了三种查询转换技术，以增强检索增强生成（RAG）系统中的检索过程：

1. 查询重写
2. 后退提示
3. 子查询分解

每种技术都旨在通过修改或扩展原始查询来提高检索信息的相关性和全面性。

## 动机

RAG系统在检索最相关的信息时经常面临挑战，特别是在处理复杂或模糊查询时。这些查询转换技术通过重新表述查询来解决这个问题，以更好地匹配相关文档或检索更全面的信息。

## 关键组件

1. 查询重写：重新表述查询使其更具体和详细。
2. 后退提示：生成更广泛的查询以获得更好的上下文检索。
3. 子查询分解：将复杂查询分解为更简单的子查询。

## 方法详情

### 1. 查询重写

- **目的**：使查询更具体和详细，提高检索相关信息的可能性。
- **实现**：
  - 使用带有自定义提示模板的GPT-4模型。
  - 接受原始查询并重新表述为更具体和详细的形式。

### 2. 后退提示

- **目的**：生成更广泛、更一般的查询，有助于检索相关的背景信息。
- **实现**：
  - 使用带有自定义提示模板的GPT-4模型。
  - 接受原始查询并生成更一般的"后退"查询。

### 3. 子查询分解

- **目的**：将复杂查询分解为更简单的子查询，以实现更全面的信息检索。
- **实现**：
  - 使用带有自定义提示模板的GPT-4模型。
  - 将原始查询分解为2-4个更简单的子查询。

## 这些方法的优势

1. **提高相关性**：查询重写有助于检索更具体和相关的信息。
2. **更好的上下文**：后退提示允许检索更广泛的上下文和背景信息。
3. **全面的结果**：子查询分解能够检索涵盖复杂查询不同方面的信息。
4. **灵活性**：每种技术都可以独立使用或组合使用，具体取决于特定用例。

## 实现细节

- 所有技术都使用OpenAI的GPT-4模型进行查询转换。
- 使用自定义提示模板来指导模型生成适当的转换。
- 代码为每种转换技术提供单独的函数，允许轻松集成到现有的RAG系统中。

## 示例用例

代码使用示例查询演示每种技术：
"气候变化对环境有什么影响？"

- **查询重写**将其扩展为包括温度变化和生物多样性等具体方面。
- **后退提示**将其概括为"气候变化的一般影响是什么？"
- **子查询分解**将其分解为关于生物多样性、海洋、天气模式和陆地环境的问题。

## 结论

这些查询转换技术为增强RAG系统的检索能力提供了强大的方法。通过以各种方式重新表述查询，它们可以显著提高检索信息的相关性、上下文和全面性。这些方法在查询可能复杂或多方面的领域特别有价值，如科学研究、法律分析或全面的事实查找任务。

# 包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [ ]:
# 安装所需的包
!pip install langchain langchain-openai python-dotenv

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

# 从.env文件加载环境变量
load_dotenv()

# 设置OpenAI API密钥环境变量
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### 1 - 查询重写：重新表述查询以改进检索。

In [2]:
re_write_llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

# 创建查询重写的提示模板
query_rewrite_template = """你是一个AI助手，负责重新表述用户查询以改进RAG系统中的检索。
给定原始查询，将其重写为更具体、详细且更可能检索到相关信息的形式。

原始查询：{original_query}

重写查询："""

query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)

# 创建用于查询重写的LLMChain
query_rewriter = query_rewrite_prompt | re_write_llm

def rewrite_query(original_query):
    """
    重写原始查询以改进检索。
    
    Args:
    original_query (str): 原始用户查询
    
    Returns:
    str: 重写的查询
    """
    response = query_rewriter.invoke(original_query)
    return response.content

### 在用例上演示

In [3]:
# 在理解气候变化数据集上的示例查询
original_query = "气候变化对环境有什么影响？"
rewritten_query = rewrite_query(original_query)
print("原始查询:", original_query)
print("\n重写查询:", rewritten_query)

Original query: What are the impacts of climate change on the environment?

Rewritten query: What are the specific effects of climate change on various ecosystems, including changes in temperature, precipitation patterns, sea levels, and biodiversity?


### 2 - 后退提示：生成更广泛的查询以获得更好的上下文检索。



In [4]:
step_back_llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)


# 创建后退提示的提示模板
step_back_template = """你是一个AI助手，负责生成更广泛、更一般的查询以改进RAG系统中的上下文检索。
给定原始查询，生成一个更一般的后退查询，可以帮助检索相关的背景信息。

原始查询：{original_query}

后退查询："""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

# 创建用于后退提示的LLMChain
step_back_chain = step_back_prompt | step_back_llm

def generate_step_back_query(original_query):
    """
    生成后退查询以检索更广泛的上下文。
    
    Args:
    original_query (str): 原始用户查询
    
    Returns:
    str: 后退查询
    """
    response = step_back_chain.invoke(original_query)
    return response.content

### 在用例上演示

In [5]:
# 在理解气候变化数据集上的示例查询
original_query = "气候变化对环境有什么影响？"
step_back_query = generate_step_back_query(original_query)
print("原始查询:", original_query)
print("\n后退查询:", step_back_query)

Original query: What are the impacts of climate change on the environment?

Step-back query: What are the general effects of climate change?


### 3- 子查询分解：将复杂查询分解为更简单的子查询。

In [6]:
sub_query_llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

# 创建子查询分解的提示模板
subquery_decomposition_template = """你是一个AI助手，负责将复杂查询分解为RAG系统的更简单子查询。
给定原始查询，将其分解为2-4个更简单的子查询，当这些子查询一起回答时，将为原始查询提供全面的响应。

原始查询：{original_query}

示例：气候变化对环境有什么影响？

子查询：
1. 气候变化对生物多样性有什么影响？
2. 气候变化如何影响海洋？
3. 气候变化对农业有什么影响？
4. 气候变化对人类健康有什么影响？"""


subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# 创建用于子查询分解的LLMChain
subquery_decomposer_chain = subquery_decomposition_prompt | sub_query_llm

def decompose_query(original_query: str):
    """
    将原始查询分解为更简单的子查询。
    
    Args:
    original_query (str): 原始复杂查询
    
    Returns:
    List[str]: 更简单子查询的列表
    """
    response = subquery_decomposer_chain.invoke(original_query).content
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries

### 在用例上演示

In [9]:
# 在理解气候变化数据集上的示例查询
original_query = "气候变化对环境有什么影响？"
sub_queries = decompose_query(original_query)
print("\n子查询:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
Original query: What are the impacts of climate change on the environment?
1. How does climate change affect biodiversity and ecosystems?
2. What are the impacts of climate change on oceanic conditions and marine life?
3. How does climate change influence weather patterns and extreme weather events?
4. What are the effects of climate change on terrestrial environments, such as forests and deserts?
